<font size="5">Alchemical Free Energy Calculation </font>

This notebook gives an example of how Free Energies are calculated for an alchemical transformation in which an Adenosine is transformed into an N6-methyladenosine. The input used are Energies computed dunring an alchemical simulations with 16 replicas, 10 ns long each.

Firstly, we compute DeltaG with the standard  Bennet Acceptance Ratio method implemented in Gromacs:


In [1]:
%%bash
module use /u/m/mbernett/my_progs/modulefiles
module load GROMACS/2019.4_bar_semiisotropic_surftens
gmx_mpi bar -f md_lam*.xvg -o -oi

md_lam0.xvg: Ignoring set 'pV (kJ/mol)'.
md_lam0.xvg: 0.0 - 10000.0; lambda = (0, 0, 0, 0, 0)
    dH/dl & foreign lambdas:
        dH/dl (mass-lambda) (500001 pts)
        dH/dl (coul-lambda) (500001 pts)
        dH/dl (vdw-lambda) (500001 pts)
        dH/dl (bonded-lambda) (500001 pts)
        dH/dl (restraint-lambda) (500001 pts)
        delta H to (0, 0, 0, 0, 0) (500001 pts)
        delta H to (0, 0.01, 0.01, 0, 0) (500001 pts)


md_lam10.xvg: Ignoring set 'pV (kJ/mol)'.
md_lam10.xvg: 0.0 - 10000.0; lambda = (0, 0.8, 0.8, 0, 0)
    dH/dl & foreign lambdas:
        dH/dl (mass-lambda) (500001 pts)
        dH/dl (coul-lambda) (500001 pts)
        dH/dl (vdw-lambda) (500001 pts)
        dH/dl (bonded-lambda) (500001 pts)
        dH/dl (restraint-lambda) (500001 pts)
        delta H to (0, 0.65, 0.65, 0, 0) (500001 pts)
        delta H to (0, 0.8, 0.8, 0, 0) (500001 pts)
        delta H to (0, 0.9, 0.9, 0, 0) (500001 pts)


md_lam11.xvg: Ignoring set 'pV (kJ/mol)'.
md_lam11.xvg: 0.0 - 

         :-) GROMACS - gmx bar, 2019.4-dev-20200518-febfc43-unknown (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mar

<font size="5">Computing Free Enegies with Binless WHAM method </font>

Energies loaded in the 2d Array correspond to energies recomputed for the demuxed concatenated trajectory for each hamiltonians corresponding to the 16 lambda intermediate. 

In [1]:
import numpy as np
import panedr
from bussilab import wham



em = "ener_lam0.edr"
df = panedr.edr_to_df(em)
df[u'Potential']


Ene2d = np.ones((len(df),16))
bias=np.ones((len(df),16))

for i in range(16):
   em = "ener_lam"+str(i)+".edr"
   df = panedr.edr_to_df(em)
   arr=np.array(df[u'Potential'])
   for j in range((len(df))):
        Ene2d[j][i]=arr[j]

In [4]:
import pickle

Ene2d=pickle.load(open("Ene2d.p","rb"))

Delta G is computed here using the binless WHAM method implemented in bussilab. Bias Energies are computed with respect to lambda=0 hamiltonian.

In [5]:
from bussilab import wham

bias=Ene2d-Ene2d[:,0][:,np.newaxis]
a =wham.wham(bias=bias, T=2.476, maxiter=10000)
-2.476*(a[u'logZ'][15]-a[u'logZ'][0])

258.24154595700566

<font size="5">Bootstrapping </font>

Here we use a Bootstrapping procedure to compute statistical error on the Free Eneriges estiamte. The energies array is resampled 200 times with respect to the 16 internal blocks which correspond to independent trajectories.

In [4]:
Nene=200
DeGs=[]

#Reading Demux indexes, they are needed to correctly count how much times the extracted trajecotries spend in each Hamiltonian system
replica_temp=np.loadtxt("../replica_temp.xvg")
replica_temp=replica_temp[:,1:]
replica_temp=replica_temp.astype(int)


for i in range(Nene):
    bias=Ene2d-Ene2d[:,0][:,np.newaxis]

    
    tr_w=np.zeros(16)
    sample=np.random.choice(16,size=16)
    for k in range(len(sample)):
        L=np.bincount(replica_temp[:,sample[k]],minlength=16)
        for j in range(len(L)):
            tr_w[j]+=L[j]
            
    bias=bias.reshape((16,-1,16))[sample,:,:].reshape((-1,16))
    a =wham.wham(bias=bias, T=2.476, traj_weight=tr_w, maxiter=10000, logZ=a.logZ)
    DeGs.append(2.476*(a[u'logZ'][15]-a[u'logZ'][0]))
    
np.std(DeGs)

0.21234460754865622